#### 0. Import basic methods (a.o. from source2.py)

In [1]:
from source2 import *
import numpy as np

In [2]:
# load basic info from the Input sheet
mapping = load_input_from_xls(filename=r'H2 Model - Input sheet (4B).xlsm')
Inputs = extract_scenario(mapping, 'Scenario1')

##### 1. prepare input data and generate objects
We are testing the following cases:
##### 1.1 Turbines

In [8]:
turbine_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Turbine')
turbine_data['depreciation_rate']=1/turbine_data['economic_lifetime']
display(turbine_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Turbine',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 1139979,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03333333333333333,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.03,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.08,
 'residual_value': 0.01}

In [9]:
Turbine = Object(**turbine_data)
# hier heb je alleen het object met inputs

In [10]:
Turbine.generate_cashflows(startyear=2023, lifecycle=29, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

year allready in list so add rather than append


##### 1.2 Foundations and cables

In [11]:
foundation_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Foundation & cable')
foundation_data['depreciation_rate']=1/foundation_data['economic_lifetime']
display(foundation_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Foundation & cable',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 971621,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03333333333333333,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.015,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.35,
 'residual_value': 0.01}

In [12]:
Foundation = Object(**foundation_data)
# hier heb je alleen het object met inputs

In [13]:
Foundation.generate_cashflows(startyear=2023, lifecycle=29, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

year allready in list so add rather than append


In [9]:
(Turbine.df+Foundation.df).tail()

,years,capex,opex,revenue
years,,,,
2048,4096,0.000000e+00,-3.111576e+08,0
2049,4098,0.000000e+00,-3.173807e+08,0
2050,4100,0.000000e+00,-3.237283e+08,0
2051,4102,0.000000e+00,-3.302029e+08,0
2052,4104,-1.565810e+09,-3.368070e+08,0


#####  2. Test values - Turbines
##### 2.1 - CAPEX values

In [16]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-697_667_148, -1_423_240_982, -1_451_705_802]}

In [17]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

CAPEX value testing in 2023: MTBS: -697667148 TUD/VO: -697667148.0
CAPEX value testing in 2024: MTBS: -1423240982 TUD/VO: -1423240981.92
CAPEX value testing in 2025: MTBS: -1451705802 TUD/VO: -1451705801.5584


##### 2.2 - OPEX values

In [6]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-116_013_366, -118_333_634, -120_700_306, -123_114_313, -125_576_599, -128_088_131, -130_649_893, -133_262_891, -135_928_149, -138_646_712, -141_419_646, -144_248_039, -147_133_000, -150_075_660, -153_077_173, -156_138_717, -159_261_491, -162_446_721, -165_695_655, -169_009_568, -172_389_760, -175_837_555, -179_354_306, -182_941_392, -186_600_220, -190_332_224, -194_138_869]}

In [7]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].opex.item(), 0)

Opex value testing in 2026: MTBS: -116013366 TUD/VO: -176453517.16589162


AssertionError: 
Arrays are not almost equal to 0 decimals
 ACTUAL: -116013366
 DESIRED: -176453517.16589162

##### 2.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-169732722.573259+624696633.5999999]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Decommissioning value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

#####  3. Test values - Foundations and cables
##### 3.1 - CAPEX values

In [14]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-594_632_052, -1_213_049_386, -1_237_310_374]}

In [15]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

CAPEX value testing in 2023: MTBS: -594632052 TUD/VO: -594632052.0
CAPEX value testing in 2024: MTBS: -1213049386 TUD/VO: -1213049386.08
CAPEX value testing in 2025: MTBS: -1237310374 TUD/VO: -1237310373.8016


##### 3.2 - OPEX values

In [16]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-98_879_912, -100_857_510, -102_874_660, -104_932_154, -107_030_797, -109_171_413, -111_354_841, -113_581_938, -115_853_576, -118_170_648, -120_534_061, -122_944_742, -125_403_637, -127_911_710, -130_469_944, -133_079_343, -135_740_930, -138_455_748, -141_224_863, -144_049_360, -146_930_348, -149_868_955, -152_866_334, -155_923_660, -159_042_134, -162_222_976, -165_467_436]}

In [17]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

Opex value testing in 2026: MTBS: -98879912 TUD/VO: -65919941.28234628


AssertionError: 
Arrays are not almost equal to 0 decimals
 ACTUAL: -98879912
 DESIRED: -65919941.28234628

##### 3.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

#####  4. Test values - AC Substation
##### 4.1 - CAPEX values

In [ ]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914_940_000, -1_866_477_600, -1_903_807_151]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

##### 4.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-319_500_591, -325_890_603, -332_408_415, -339_056_583, -345_837_715, -352_754_469, -359_809_558, -367_005_749, -374_345_864, -381_832_782, -389_469_437, -397_258_826, -405_204_003, -413_308_083, -421_574_244, -430_005_729, -438_605_844, -447_377_961, -456_325_520, -465_452_030, -474_761_071, -484_256_292, -493_941_418, -503_820_246, -513_896_651, -524_174_584, -534_658_076]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 4.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

#####  5. Test values - DC Collection Cable
##### 5.1 - CAPEX values

In [ ]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914_940_000, -1_866_477_600, -1_903_807_151]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

##### 5.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-319_500_591, -325_890_603, -332_408_415, -339_056_583, -345_837_715, -352_754_469, -359_809_558, -367_005_749, -374_345_864, -381_832_782, -389_469_437, -397_258_826, -405_204_003, -413_308_083, -421_574_244, -430_005_729, -438_605_844, -447_377_961, -456_325_520, -465_452_030, -474_761_071, -484_256_292, -493_941_418, -503_820_246, -513_896_651, -524_174_584, -534_658_076]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 5.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

#####  6. Test values - DCAC Converter
##### 6.1 - CAPEX values

In [ ]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914_940_000, -1_866_477_600, -1_903_807_151]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

##### 6.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-319_500_591, -325_890_603, -332_408_415, -339_056_583, -345_837_715, -352_754_469, -359_809_558, -367_005_749, -374_345_864, -381_832_782, -389_469_437, -397_258_826, -405_204_003, -413_308_083, -421_574_244, -430_005_729, -438_605_844, -447_377_961, -456_325_520, -465_452_030, -474_761_071, -484_256_292, -493_941_418, -503_820_246, -513_896_651, -524_174_584, -534_658_076]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 6.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

#####  7. Test values - Mainland
##### 7.1 - CAPEX values

In [ ]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914_940_000, -1_866_477_600, -1_903_807_151]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

##### 7.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-319_500_591, -325_890_603, -332_408_415, -339_056_583, -345_837_715, -352_754_469, -359_809_558, -367_005_749, -374_345_864, -381_832_782, -389_469_437, -397_258_826, -405_204_003, -413_308_083, -421_574_244, -430_005_729, -438_605_844, -447_377_961, -456_325_520, -465_452_030, -474_761_071, -484_256_292, -493_941_418, -503_820_246, -513_896_651, -524_174_584, -534_658_076]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 7.3 - Decommissioning + divestment

In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)